- Image scale normalization
- blurring ✅
- sharpness
- shifting ✅
- merging?

In [410]:
import matplotlib.image as mimg
import matplotlib.pyplot as plt
import numpy as np
import math

In [411]:
img1 = mimg.imread('Input Images/Spiral.jpg')
img2 = mimg.imread('Input Images/Oogway_forest.jpg')

In [418]:
# other functions

# overlay dependency
def _avg_pixel_weighted(channel_val1, weight1, channel_val2, weight2):
    adjusted_weight_total = weight1 + weight2
    return ( (channel_val1 * weight1 + channel_val2 * weight2)/adjusted_weight_total)


# blur, pixelate dependency
def _avg_color_grid(img, grid_size, start_pixel):

    r_total, g_total, b_total=0, 0, 0
    x_dim, y_dim = start_pixel
    counter = grid_size**2

    padding = grid_size//2

    for i in range(-padding, padding + 1):
        for j in range(-padding, padding + 1):

            r_total += img.item((x_dim + i, y_dim + j, 0))
            g_total += img.item((x_dim + i, y_dim + j, 1))
            b_total += img.item((x_dim + i, y_dim + j, 2))

    return np.array((r_total / counter, g_total / counter, b_total / counter))


In [413]:
# OVERLAY

def overlay(img1, img1_weight, img2, img2_weight):
    
    max_x_size = max(img1.shape[0], img2.shape[0])
    max_y_size = max(img1.shape[1], img2.shape[1])

    new_img_arr = np.empty((max_x_size, max_y_size, 3))

    for i in range(max_x_size):
        for j in range(max_y_size):
            for k in range(3):

                new_img_arr[i, j, k] = _avg_pixel_weighted(img1.item((i, j, k)), img1_weight, img2.item((i, j, k)), img2_weight) / 255  # 4 sec

    return new_img_arr


In [420]:
# PIXELATE

def pixelate(img, pixelate_amount):
    # 2 - 17.8s
    # 3 - 24.6s
    # 5 - ~47s
    # 10 - 2m 37.5s

    height = img.shape[0]
    width = img.shape[1]

    padding = pixelate_amount//2

    new_img_arr = np.empty((height, width, 3))

    for i in range(padding, height - pixelate_amount, pixelate_amount):
        for j in range(padding, width - pixelate_amount, pixelate_amount):
            color = _avg_color_grid(img, pixelate_amount, (i, j)) / 255
            
            for _i in range(pixelate_amount):
                for _j in range(pixelate_amount):
                    new_img_arr[i + _i, j + _j] = color

    return new_img_arr


In [415]:
# BLUR

def blur(img, blur_size = 3):
    # blur_size = int(input("Blur size: "))
    # 2 - 17.8s
    # 3 - 24.6s
    # 5 - ~47s
    # 10 - 2m 37.5s

    padding = blur_size//2

    height = img.shape[0]
    width = img.shape[1]

    new_img_arr = np.empty((height, width, 3))

    for i in range(padding, height - padding):
        for j in range(padding, width - padding):

            new_img_arr[i, j] = _avg_color_grid(img, blur_size, (i, j)) / 255



    return new_img_arr


In [416]:
# SHIFTING
import random

def shift(img, operation, start_val_increment=1.6, shift_x=10, shift_y=10, start_offset=0): 
    # operation = int(input("what operation should be performed? (0:linear, 1:sin, 2:cos, 3:tan, 4:random)  "))
    multiplicant = 0

    start_val = 0

    height = img.shape[0]
    width = img.shape[1]

    new_img_arr = np.empty((height, width, 3))

    for i in range(height):
        for j in range(width):
            for k in range(3):
                if operation == 4:
                    multiplicant = random.random() * 2 - 1  

                new_img_arr[i, j, k] = img.item((
                    ((int(multiplicant * shift_x)) + i + start_offset) %
                    height,
                    ((int(multiplicant * shift_y)) + j + start_offset) %
                    width, k)) / 255

        multiplicant = start_val if operation == 0 else math.sin(start_val) if operation == 1 else math.cos(start_val) if operation == 2 else math.tan(start_val) if operation == 3 else 0

        start_val+=start_val_increment # for sin or cos, min and max shift is from 0 to 1.5707 (π/2)

    return new_img_arr


In [422]:
# plt.imshow(shift(img2))

# plt.imsave("Output Images/Overlay/Overlay 3-1.jpg", overlay(img1, img2))
# plt.imsave("Output Images/Blur/Blur padding.jpg", blur(img1))
plt.imsave("Output Images/Pixelate/Pixelate padding.png", pixelate(img1, 25))
# plt.imsave("Output Images/Distort/Random.png", shift(img2, 4))